# Julia for FEM: ``Ferrite.jl``
Ferrite is a finite element toolbox that provides functionalities to implement finite element analysis in Julia. The aim is to be general and to keep mathematical abstractions. The main functionalities of the package include:
- Facilitate integration using different quadrature rules.
- Define different finite element interpolations.
- Evaluate shape functions, derivatives of shape functions etc. for the different interpolations and quadrature rules.
- Evaluate functions and derivatives in the finite element space.
- Generate simple grids.
- Export grids and solutions to VTK.

This package used to be called ``JuAFEM.jl``.

## Installation
You can install Ferrite from the Pkg REPL (press ``]`` in the Julia REPL to enter ``pkg>`` mode):
```
pkg> add Ferrite
```

## Documentation & Tutorials

## Integration with ``Gmsh``

```julia
using Ferrite, FerriteGmsh

saved_file_to_grid("path/to/model.msh")
```

## Visualization

# Example

In [11]:
V = 245e3 * sqrt(2/3);

200041.66232729287

In [1]:
using Ferrite, FerriteGmsh
using SparseArrays

In [24]:
grid = saved_file_to_grid("geo/gridap_example.msh");

Info    : Reading 'geo/gridap_example.msh'...
Info    : 21 entities
Info    : 26724 nodes
Info    : 54640 elements
Info    : Done reading 'geo/gridap_example.msh'


In [103]:
dim = 2
ip_fe  = Lagrange{dim, RefTetrahedron, 2}()
ip_geo = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2)
cellvalues = CellScalarValues(qr, ip_fe, ip_geo);

dh   = DofHandler(grid);
push!(dh, :V, 1, ip_fe)
close!(dh)

DofHandler
  Fields:
    :V, interpolation: Lagrange{2, RefTetrahedron, 2}(), dim: 1
  Dofs per cell: 6
  Total dofs: 106817

In [104]:
ch = ConstraintHandler(dh);

dbc_V = Dirichlet(
    :V,
#    union(getfaceset(grid, "D1"), getfaceset(grid, "D1p")),
    getfaceset(grid, "D1"),
    (x,t) -> V;
);
dbc_0 = Dirichlet(
    :V,
    getfaceset(grid, "D2"),
#    union(getfaceset(grid, "D2"), getfaceset(grid, "D2p")),
    (x,t) -> 0;
)

add!(ch, dbc_V)
add!(ch, dbc_0)

close!(ch)
update!(ch, 0.0);

In [105]:
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # Add contribution to Ke
                Ke[i, j] += (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Compute element contribution
        assemble_element!(Ke, fe, cellvalues)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [106]:
K = create_sparsity_pattern(dh);
K, f = assemble_global(cellvalues, K, dh);

apply!(K, f, ch)
u = K \ f;

## Post-processing

In [116]:
function compute_E(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}
    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    E = [Ferrite.Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        E_cell = E[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        ae = a[cell_dofs]
        reinit!(cellvalues, cell)

        for E_point in 1:nqp
            E_qp = - function_gradient(cellvalues, E_point, ae)
            push!(E_cell, E_qp)
        end
    end
    return E
end

E_gp = compute_E(cellvalues, dh, u);
E = collect(mean(norm.(E_gp[i])) for i = 1:getncells(dh.grid))

projector = L2Projector(ip, grid);
E_proj = project(projector, E_gp, qr; project_to_nodes = false);

In [133]:
vtk_grid("ferrite_example", dh) do vtk
    vtk_point_data(vtk, dh, u)
    vtk_cell_data(vtk, E, "E")
end

1-element Vector{String}:
 "ferrite_example.vtu"

## Visualization

In [31]:
using gmsh
using CairoMakie
using GeometryBasics

CairoMakie.activate!(type = "svg")

In [40]:
# Obtain a GeometryBasics.Mesh object suitable for plotting with Makie
function get_node_mesh()
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nNode = length(node_ids)

    eType, eTag, eConn = gmsh.model.mesh.getElements(2);
    nEl = length(eTag[1])

    points   = zeros(Point{2, Float64}, nNode);          # Array of vertex coordinates (x,y)
    trif     = zeros(TriangleFace{Int}, nEl);  # Array of triangular faces (n1, n2, n3)
    
    for n in 1:nNode
        points[node_ids[n]] = Point{2, Float64}(node_coord[3*(n-1) + 1], node_coord[3*(n-1) + 2])
    end
    for e in 1:nEl
        trif[e] = (eConn[1][3*(e-1)+1], eConn[1][3*(e-1)+2], eConn[1][3*(e-1)+3])
    end
    
    msh = GeometryBasics.Mesh(points, trif);
    return msh;
end

get_node_mesh (generic function with 1 method)

In [41]:
gmsh.initialize()
gmsh.open("geo/gridap_example.msh")

msh = get_node_mesh()

f, ax, pl = mesh(msh, color = u, colormap = :lighttest, shading = false)
#wireframe!(ax, msh, color=(:black), linewidth=1, transparency=false)
ax.aspect = AxisAspect(1)
current_figure()

Info    : Reading 'geo/gridap_example.msh'...
Info    : 21 entities
Info    : 26724 nodes
Info    : 54640 elements
Info    : Done reading 'geo/gridap_example.msh'


TODO: re-order nodes to ensure this looks ok